### Variablen

- Veranstaltungen (Weihnachtsmarkt, Dom (Freimärkte), Fußballspiele, Sonstige) -> Luisa

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import csv
import re

In [ ]:
# Webseite laden
jahrmaerktenord_url = 'https://example.com'
response = requests.get(jahrmaerktenord_url))

# HTML parsen
soup = bs(response.text, 'html.parser')

# Beispiel: Alle Links extrahieren
links = soup.find_all('a')  # Alle <a>-Tags extrahieren
for link in links:
    print(link.get('href'))  # Nur die URLs der Links anzeigen

# Beispiel: Text aus einem bestimmten Element extrahieren
specific_data = soup.find('h1')  # Text aus dem ersten <h1>-Tag extrahieren
if specific_data:
    print(specific_data.text)
else:
    print(f"Fehler beim Abrufen der Webseite. Status Code: {response.status_code}")



In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from datetime import datetime

def fetch_and_extract_data():
    # URL der Webseite
    base_url = 'https://jahrmarktnord.de/category/archiv'  # Basis-URL für die Seiten
    event_data = []
    current_page = 1  # Startseite für die Paginierung

    while True:
        url = f"{base_url}/page/{current_page}/"
        print(f"Durchsuche Seite: {url}")
        
        # HTTP-Anfrage senden und die Webseite abrufen
        response = requests.get(url)
        
        # Überprüfen, ob die Anfrage erfolgreich war
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Alle Veranstaltungseinträge suchen
            events = soup.find_all('article')
            print(f"Anzahl der Veranstaltungen auf Seite {current_page}: {len(events)}")

            if not events:
                print("Keine Veranstaltungen auf dieser Seite gefunden. Suche beendet.")
                break  # Keine Veranstaltungen gefunden, Stoppe die Schleife

            # Durch alle Veranstaltungen iterieren
            for event in events:
                # Suche nach dem Titel der Veranstaltung (z. B. "Geesthacht Herbstmarkt 2019")
                title_tag = event.find('h3', class_='entry-title')
                if title_tag:
                    title = title_tag.text.strip()
                    print(f"Veranstaltung gefunden: {title}")
                    
                    # Prüfen, ob "Kiel" im Titel ist
                    if 'Kiel' in title:
                        print(f"    Titel mit 'Kiel' gefunden: {title}")

                        # Suche nach dem Text, der den Zeitraum enthält
                        excerpt_tag = event.find('div', class_='mh-excerpt')
                        if excerpt_tag:
                            # Den Text extrahieren, der den Zeitraum enthält
                            excerpt_text = excerpt_tag.text.strip()
                            print(f"    Zeitraum-Text: {excerpt_text}")

                            # RegEx: Zeitraum im Format (tt.mm-tt.mm.jjjj) extrahieren
                            match = re.search(r'(\d{2}\.\d{2})-(\d{2}\.\d{2}\.\d{4})', excerpt_text)
                            if match:
                                start_date = match.group(1)
                                end_date = match.group(2)
                                event_data.append([title, start_date, end_date])
                            else:
                                # Versuche, den Zeitraum im weniger präzisen Format zu extrahieren
                                match = re.search(r'(\d{2}\.\d{2})-(\d{2}\.\d{2})', excerpt_text)
                                if match:
                                    start_date = match.group(1)
                                    end_date = match.group(2)
                                    
                                    # Wenn kein Jahr angegeben, versuche, das Jahr aus dem Titel oder dem Datum zu extrahieren
                                    year_from_title = extract_year_from_title(title)
                                    if year_from_title:
                                        end_date = end_date + '.' + year_from_title  # Jahr ergänzen
                                    else:
                                        # Wenn Jahr nicht im Titel vorhanden, versuche das aktuelle Jahr zu verwenden
                                        current_year = str(datetime.now().year)
                                        end_date = end_date + '.' + current_year  # Jahr annehmen
                                    event_data.append([title, start_date, end_date])
                                else:
                                    # Wenn kein passender Zeitraum gefunden wird, nur den Titel speichern
                                    print("    Kein gültiger Zeitraum gefunden.")
        else:
            print(f"Fehler beim Abrufen der Seite {current_page}. Status Code: {response.status_code}")
            break

        current_page += 1  # Nächste Seite aufrufen

    # Wenn Daten vorhanden sind, schreibe sie in eine CSV-Datei
    if event_data:
        write_to_csv(event_data)
    else:
        print("Keine relevanten Veranstaltungen gefunden.")

def extract_year_from_title(title):
    """
    Diese Funktion extrahiert das Jahr aus dem Titel der Veranstaltung, falls vorhanden.
    Beispiel: 'Weihnachtsmarkt Kiel 2022' -> '2022'
    """
    match = re.search(r'(\d{4})', title)
    if match:
        return match.group(1)
    return None

def write_to_csv(data):
    # CSV-Datei erstellen und Daten speichern
    with open('veranstaltungen_kiel.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Veranstaltung', 'Startdatum', 'Enddatum'])  # Header
        writer.writerows(data)
        print("Daten wurden erfolgreich in die CSV-Datei geschrieben!")

# Aufruf der Funktion
fetch_and_extract_data()


Durchsuche Seite: https://jahrmarktnord.de/category/archiv/page/1/
Anzahl der Veranstaltungen auf Seite 1: 10
Veranstaltung gefunden: Leer Gallimarkt 2024, Aufbau Di. 08.10.
Veranstaltung gefunden: Neukloster/Buxtehude Pfingstmarkt 2024
Veranstaltung gefunden: Flensburg Frühjahrsmarkt 2024
Veranstaltung gefunden: Heide Frühjahrsmarkt 2024
Veranstaltung gefunden: Oldenburg Kramermarkt 2023, Aufbau Do. 21.09.
Veranstaltung gefunden: Emden Ostermarkt 2023
Veranstaltung gefunden: Rheine Kirmes 2022 – Abbau
Veranstaltung gefunden: Bruchhausen – Vilsen  Brokser Heiratsmarkt 2022 (Premiere „Mike`s Pitstop“)
Veranstaltung gefunden: Heider Frühjahrsmarkt 2022 erneut abgesagt!
Veranstaltung gefunden: Hamburg Winterdom Aufbau 29 Oktober 2021
Durchsuche Seite: https://jahrmarktnord.de/category/archiv/page/2/
Anzahl der Veranstaltungen auf Seite 2: 10
Veranstaltung gefunden: Stralsund Weihnachtsmarkt 2019
Veranstaltung gefunden: Rostock Weihnachtsmarkt 2019
Veranstaltung gefunden: Grabow/Meckl. Mar